In [1]:
from src.tools import MMseqs
import pandas as pd
import numpy as np
from src.files import XMLFile

%load_ext autoreload 
%autoreload 2

In [19]:
df = pd.read_csv('../data/sprot.csv', low_memory=False, index_col=0)
df = df[df.domain == 'Bacteria']
df = df[df.non_terminal_residue.isnull()]

In [25]:
mmseqs = MMseqs()
df = mmseqs.cluster(df, job_name='sprot_95', sequence_identity=0.95, reps_only=True, overwrite=False)
df = mmseqs.cluster(df, job_name='sprot_20', sequence_identity=0.2, reps_only=False, overwrite=False)
mmseqs.cleanup()


In [26]:
len(df)

191078

In [36]:
# Want to look at start codon frequencies across the SwissProt sequences. 

def table_1(df:pd.DataFrame, path:str=None):
    df['start_aa'] = [seq[0] for seq in df.seq]
    # return df[df.start_aa != 'M']
    return df

x = table_1(df.copy())
x[x.start_aa == 'T']

x.start_aa.value_counts()

start_aa
M    190802
A       104
G        21
S        21
E        20
Q        18
T        14
K        10
V         9
I         9
Y         8
D         8
L         7
N         7
C         6
P         5
F         5
X         2
R         1
H         1
Name: count, dtype: int64